# Instalar Telegram API

In [0]:
!pip install python-telegram-bot==5.3.0

# Configurar Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Seleccionar directorio de trabajo

In [0]:
%cd 'drive/My Drive/classify_bot'
!pwd
!ls

# Librerias

In [0]:
%tensorflow_version 1.x
import requests
import tensorflow as tf
import sys
import os
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
import logging

print("Librerías importadas correctamente")

# Classify

In [0]:
def classify(image_path):
    # Read the image_data
    image_data = tf.io.gfile.GFile(image_path, 'rb').read()

    # Loads label file, strips off carriage return
    label_lines = [line.rstrip() for line in tf.io.gfile.GFile("tf_files/retrained_labels.txt")]

    # Unpersists graph from file
    with tf.io.gfile.GFile("tf_files/retrained_graph.pb", 'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

    with tf.compat.v1.Session() as sess:
        # Feed the image_data as input to the graph and get first prediction
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        
        predictions = sess.run(softmax_tensor, \
                {'DecodeJpeg/contents:0': image_data})
        
        # Sort to show labels of first prediction in order of confidence
        top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
        
        result = None
        a=sorted(predictions[0], reverse=True)
        print(a)
        print(a[0])
        for node_id in top_k:
            human_string = label_lines[node_id]
            accuracy = predictions[0][node_id]
            
            print('%s (score = %.6f)' % (human_string, accuracy))
            if accuracy == a[0]:
              if accuracy>=0.4:
                result = human_string
                if result=="bar":
                  result=" Holaa....Soy Doctor mente ...... Si tu aceptas podria ayudarte a saber si tu padeces o tienes principios de  Alzheimer aceptas "
                elif result=="foo":
                  result="Empecemos no se diga mas :)"
                indicator=1
                break
              else:
                result="por el momento lo reconsideras aqui estare siempre para atenderte y recomendarte que puede ser mejor para ti..Te deseo bonito dia y nos vemos Pronto" 
    return result

    print("Listo")

# Token de telegram

In [0]:
# ClassifyImagesBot
token = '1175565197:AAEF6t72lskYrKHwjtFqI3IWOxBFvtUj-Bw'

print("token registrado")

# Configuración del bot


In [0]:

# Enable logging
try:
    logging.basicConfig(
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
    logger = logging.getLogger(__name__)
except Exception as e:
    print("Error logging {}".format(e.args))

def start(bot, update):
    try:
        username = update.message.from_user.username
        message = "Holaaa " + username
        update.message.reply_text(message)
    except Exception as e:
        print("Error start {}".format(e.args))


def help(bot, update):
    try:
        username = update.message.from_user.username
        update.message.reply_text('Hello {}, please send a image for classify'.format(username))
    except Exception as e:
        print("Error help {}".format(e.args))

def analize(bot, update):
    try:
        message = "la imagen solo es por si quieres que te conozca"
        update.message.reply_text(message)
        photo_file = bot.getFile(update.message.photo[-1].file_id)
        id_user = update.message.from_user.id
        id_file = photo_file.file_id
        id_analisis = str(id_user) + "-" + str(id_file)
        filename = os.path.join('downloads/', '{}.jpg'.format(id_analisis))
        photo_file.download(filename)
        message = "te atiendo"
        update.message.reply_text(message)
        result = classify(filename)
        update.message.reply_text(result)
        print(".......e")
    except Exception as e:
        print("Nop {}".format(e.args))
text_usuar=[]
text_bot=[]     

def echo(bot, update):
    try:    
        id_user = update.message.from_user.id
        texto=update.message.text
        usuario=update.message.from_user['first_name']
        print(texto)
        print(id_user)

        time.sleep(1)

        if "hola" in texto or "HOLA" in texto or "Hola" in texto or "Hello" in texto or "hi" in texto or "Hi" in texto or "Hello" in texto:
          regreso="Holaa....Soy Doctor mente ...... Si tu aceptas podria ayudarte a saber si tu padeces o tienes principios de  Alzheimer aceptas "
        elif texto=="Acepto" or texto=="Por supuesto" or texto=="claro" or texto=="Si Doctor mente " or "SI" in texto or "Si por favor" in texto or "acepto" in texto :
          regreso="Empecemos no se diga mas :) "
        elif texto=="no" or texto=="mas tarde" in texto or "posponer" in texto or "mejor despues" in texto or "Despues" in texto or "Posponer mas tarde" in texto:
          regreso="por el momento lo reconsideras aqui estare siempre para atenderte y recomendarte que puede ser mejor para ti..Te deseo bonito dia y nos vemos Pronto"
        elif "Bueno" in texto or "Esta Bien" in texto or "Estoy listo " in texto or "empecemos" in texto:
          regreso="1 .¿ En que año estamos?"
        elif "en el año 2020" in texto or "2020" in texto or "2019" in texto or "en 2020" in texto or "estamos en el año 2020" in texto:
          regreso="2. ¿en que año ocurrio el terremoto mas fuerte en Mexico antes del año 2000?"
        elif "en el año 1985" in texto or "1980" in texto or "1985" in texto or "1987" in texto or "nose" in texto or "1983" in texto or "1984" in texto or "19 de septiembre de 1985" in texto:
          regreso="3. ¿Cuántas monedas de cinco centavos hay en 60 centavos?"
        elif "seis" in texto or "6" in texto or "12" in texto or "nose" in texto or "doce" in texto:
          regreso="4. ¿En qué se parecen un reloj de pulsera y una regla? Escriba en qué se parecen. …"
        elif "en que los dos miden" in texto or "nose" in texto or "ni idea" in texto or "lo largo" in texto or "en los 2" in texto:
          regreso="5 .¿Cuál es el octavo mes del año?"
        elif "octubre" in texto or "septiembre" in texto or "agosto" in texto or "julio" in texto or "noviembre" in texto:
          regreso="6 .¿Cuántos gramos hay en un cuarto de kilo?"
        elif "125 gramos" in texto or "250 gramos" in texto or "300 gramos" in texto or "nose" in texto:
          regreso="7 .¿En que continente esta Ecuador?"
        elif "america" in texto or "asia" in texto or "africa" in texto or "continente americano" in texto:
          regreso="8. ¿Qué son los humanos: omnívoros, herbívoros o carnívoros?"
        elif "Herbívoros" in texto or "Omnívoros" in texto or "herbívoros o carnívoros" in texto or "herbívoros" in texto or "carnívoros" in texto or "Omnívoros y Herbívoros" in texto:
          regreso=" 9.¿Cuál es el océano más grande del mundo?"
        elif "Atlantico" in texto or "Pacifico" in texto or "Indico" in texto or "nose" in texto:
          regreso="10. ¿Si 50 es el 100%, ¿cuánto es el 90%?"
        elif "El resultado de esta regla de tres es 45" in texto or "45" in texto or "90" in texto or "40" in texto or "nose" in texto or "48" in texto:
          regreso="11. ¿Cuál es tercer planeta en el sistema solar?"
        elif "El planeta Tierra" in texto or "marte" in texto or "venus" in texto or "nose" in texto:
            regreso="para las siguientes preguntas favor de contestarme con: (siempre)(constantemente)(algunas veces)(regularmente)(nunca)(no lo recuerdo)¿te parece bien?"
        elif "SI" in texto or "si" in texto or "Si" in texto or "si me parece bien" in texto or "esta bien" in texto:
            regreso="1. ¿con que frecuencia tienes dificultad para recordar eventos, planificar o resolver problemas?"
        elif "siempre" in texto or "constantemente" in texto or "algunas veces" in texto or "regularmente" in texto or "nunca" in texto or "no lo recuerdo" in texto:
            regreso="2. ¿con que frecuencia tienes dificultad para recordar eventos, planificar o resolver problemas?"
        elif "siempre" in texto or "constantemente" in texto or "regularmente" in texto or "algunas veces" in texto or "nunca" in texto or "no lo recuerdo" in texto:
            regreso="3. ¿Con que Regularidad tienes Confusión con respecto a los lugares o el paso del tiempo? "
        elif "siempre" in texto or "constantemente" in texto or "regularmente" in texto or "algunas veces" in texto or "nunca" in texto or "no lo recuerdo" in texto:
            regreso="4 .¿Que tan frecuente  tienes Problemas para completar tareas diarias en el hogar o en el trabajo?"
        elif "siempre" in texto or "constantemente" in texto or "regularmente" in texto or "algunas veces" in texto or "nunca" in texto or "no lo recuerdo" in texto:
            regreso="5. ¿ Con que regularidad tienes  Problemas de lenguaje, como por ejemplo, problemas para encontrar palabras o vocabulario reducido?"
        elif "siempre" in texto or "constantemente" in texto or "regularmente" in texto or "algunas veces" in texto or "nunca" in texto or "no lo recuerdo" in texto:
            regreso="6. ¿Que tan frecuente  sueles perderte en la conversación mientras hablas con alguien?"
        elif "siempre" in texto or "constantemente" in texto or "regularmente" in texto or "algunas veces" in texto or "nunca" in texto or "no lo recuerdo" in texto:
            regreso="pregunta especial ¿en que año ocurrio el terremoto mas fuerte en Mexico antes del año 2000?"
        elif "en el año 1985" in texto or "1980" in texto or "1985" in texto or "1987" in texto or "nose" in texto or "1983" in texto or "1984" in texto or "19 de septiembre de 1985" in texto:
            regreso="2-4 errores de Alzheimer Leve          5-7 errores Alzheimer Moderado          8-11 errores Usted tiene Graves problemas de Alzheimer ¿conteste cuantos errores tuvo? especificame con numero de las opciones ejemplo: 2-4 en caso de heber sacado o 5-7 o la ultima 8-11"
        elif "1" in texto or "2" in texto or "3" in texto or "4" in texto or "2-4" in texto:
            regreso="Enfermedad de Alzheimer leve (etapa temprana) Caracteristicas: Problemas para encontrar la palabra o el nombre correctos Dificultad para recordar nombres cuando se presenta a personas nuevas Dificultad para realizar tareas en entornos sociales o laborales. Olvidarse de algo que acaba de leer Perder o traspapelar un objeto valioso Tener más problemas para planificar u organizar ESPERO HABERTE AYUDADO"
        elif "5" in texto or "6" in texto or "7" in texto or "5-7" in texto:
            regreso="Si la respuesta fue 5-7 Enfermedad de Alzheimer moderada (etapa media) Caracteristicas: Olvidarse eventos o información de la historia personal Sentirse malhumorado o retraído, especialmente en situaciones social o mentalmente exigentes No poder recordar la dirección o el número de teléfono propios, o la escuela o universidad donde se graduó Confusión sobre la ubicación y fecha actuales Necesitar ayuda para elegir la ropa adecuada para la temporada o la ocasión Problema para controlar la necesidad de ir al baño Cambios en los patrones de sueño, como dormir durante el dia ESPERO HABERTE AYUDADO"
        elif "8" in texto or "9" in texto or "10" in texto or "11" in texto or "8-11" in texto:
            regreso="Si la respuesta es de 8-11 Enfermedad de Alzheimer grave (etapa final) Necesitar asistencia todo el tiempo con las actividades diarias y el cuidado personal Perder la noción de experiencias recientes y los hechos que las rodean Experimentar cambios en las capacidades físicas, incluida la capacidad de hablar, sentarse y, eventualmente, tragar Tener más dificultad para comunicarse Volverse vulnerables a infecciones, especialmente neumonía ESPERO HABERTE AYUDADO"
        elif "muy buena informacion" in texto or "esta bien" in texto or "ok" in texto or "mas informacion" in texto or "thanks" in texto or "gracias" in texto:
            regreso="Que es falta de memoria en corto plazo yo como tu doctor virtual  te recomiendo que vayas con un doctor de forma presencial para que pueda atender tu caso y que te recete un medicamento apropiado ademas entre en una dieta balanceada y especializada entra a este link para mas informacion http://www.alzheimerferrolterra.com/images/pdf/recomendaciones_para_la_atencion_de_personas_con_dependencia.pdf y RECUERDA ESTA FRASE ----RECIBIR UNA SONRISA SUYA NO TIENE PRECIO, AUNQUE NO SEPAS QUIEN SOY...SOLO QUIERO QUE SEPAS QUE YO SIEMPRE TE VOY A QUERRER Y CUIDAR---- ahora te explicare el test de con que frecuencia te suceden las situaciones anteriormente preguntadas... aceptas"
        elif "si" in texto or "claro" in texto or "acepto" in texto or "esta bien" in texto or "bueno" in texto or "ok" in texto:
            regreso="si tus respuestas fueron (siempre), (constantemente), (regularmente) visita un doctor con la ayuda de un familiar cercano que este pendiente de ti....y si tus respuestas fueron (algunas veces), (nunca) y (no lo recuerdo) toma medidas te recomiendo leer la informacion que te comparti y recuerda que estoy para atenderte"
        elif "gracias" in texto or "Gracias" in texto or "GRACIAS" in texto or "MUCHAS GRACIAS" in texto or "Muchas Gracias" in texto:
            regreso="No hay de que sabes que cuentas con mi ayuda y asesoria"
        else:
            regreso="espero verte pronto adios..."
          
        update.message.reply_text(regreso)
        if(len(text_usuar)==0):
          text_usuar.append(texto)
          text_bot.append(regreso)   
        elif(len(text_usuar)==1):
          text_usuar.append(texto)
          text_usuar.pop(0)
          text_bot.append(regreso)
          text_bot.pop(0)
          print(text_usuar)
          print(text_bot)
          print(update.message.from_user['first_name'])
          print("Un momento por favor...")
    except Exception as e:
        print("Error echo {}".format(e.args))

def error(bot, update, error):
    try:
        logger.warn('Update "%s" caused error "%s"' % (update, error))
    except Exception as e:
        print("Error error {}".format(e.args))

def main():
    try:
        updater = Updater(token)
        # Get the dispatcher to register handlers
        dp = updater.dispatcher

        # on different commands - answer in Telegram
        dp.add_handler(CommandHandler("start", start))
        dp.add_handler(CommandHandler("help", help))

        # on noncommand detect the document type on Telegram
        dp.add_handler(MessageHandler(Filters.text, echo))
        dp.add_handler(MessageHandler(Filters.photo, analize))

        # log all errors
        dp.add_error_handler(error)

        # Start the Bot
        updater.start_polling()
        print('ClassifyImagesBot ready')
        updater.idle()
    except Exception as e:
        print("Error main {}".format(e.message))

def alzheimerferrolterra(bot, update):
    file = open('./alzheimerferrolterra.url','rb')
    update.message.reply_document(file)

print("listo")


# Ejecutar el Bot

In [0]:
if __name__ == '__main__':
    try:
        main()
    except Exception as e:
        print("Error name: {}".format(e.args))